<a href="https://colab.research.google.com/github/fleithpi/PROGRAMACAO-ORIENTADA-A-OBJETOS/blob/PROGRAMACAO-ORIENTADA-A-OBJETOS/Gest%C3%A3o_de_Eventos_com_interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import uuid


class Evento:
    def __init__(self, nome, data, local, descricao, capacidade):
        self.id = str(uuid.uuid4())
        self._nome = nome.strip()
        self._data = data.strip()
        self._local = local.strip()
        self._descricao = descricao.strip()
        self._capacidade_maxima = capacidade
        self._participantes = []
        self._ingressos = []


    @property
    def nome(self):
        return self._nome

    @property
    def data(self):
        return self._data

    @property
    def local(self):
        return self._local

    @property
    def descricao(self):
        return self._descricao

    @property
    def capacidade_maxima(self):
        return self._capacidade_maxima

    @property
    def participantes(self):
        return self._participantes

    @property
    def ingressos(self):
        return self._ingressos


    @nome.setter
    def nome(self, novo_nome):
        self._nome = novo_nome.strip()

    @data.setter
    def data(self, nova_data):
        self._data = nova_data.strip()

    @local.setter
    def local(self, novo_local):
        self._local = novo_local.strip()

    @descricao.setter
    def descricao(self, nova_descricao):
        self._descricao = nova_descricao.strip()

    @capacidade_maxima.setter
    def capacidade_maxima(self, nova_capacidade):
        self._capacidade_maxima = nova_capacidade

    # métodos
    def add_participante(self, participante):
        if len(self.participantes) >= self.capacidade_maxima:
            print("Sold out. Evento lotado.")
            return False
        self.participantes.append(participante)
        return True

    def gerar_ingresso(self, ingresso):
        self.ingressos.append(ingresso)

    def ocupacao(self):
        return len(self.participantes)

    def __str__(self):
        return (f"Evento: {self.nome} (id: {self.id}) - Data: {self.data} - Local: {self.local}")

In [22]:

class Participante:
    def __init__(self, nome, cpf, email, telefone, evento):
        self._nome = nome.strip()
        self._cpf = cpf.strip()
        self._email = email.strip()
        self._telefone = telefone.strip()
        self._evento = evento

    @property
    def nome(self):
        return self._nome

    @property
    def cpf(self):
        return self._cpf

    @property
    def email(self):
        return self._email

    @property
    def telefone(self):
        return self._telefone

    @property
    def evento(self):
        return self._evento

    def __str__(self):
        evnome = self.evento.nome if isinstance(self.evento, Evento) else str(self.evento)
        return (f"Participante: {self.nome} (CPF: {self.cpf}) - Email: {self.email} - "
                f"Telefone: {self.telefone} - Evento: {evnome}")

In [23]:

class Ingresso:
    def __init__(self, id_evento):
        self.id = str(uuid.uuid4())
        self._id_evento = id_evento
        self._status = "disponivel"
        self._comprador = None

    @property
    def status(self):
        return self._status

    @status.setter
    def status(self, novo_status):
        self._status = novo_status

    @property
    def comprador(self):
        return self._comprador

    @comprador.setter
    def comprador(self, novo_comprador):
        self._comprador = novo_comprador

    @property
    def id_evento(self):
        return self._id_evento

    def vender(self, comprador):
        if self.status != "disponivel":
            print("Ingresso não está disponível.")
            return False
        self.status = "vendido"
        self.comprador = comprador
        return True

    def cancelar(self):
        if self.status == "cancelado":
            print("O ingresso já está cancelado.")
            return False
        self.status = "cancelado"
        self.comprador = None
        return True

    def __str__(self):
        comprador_txt = f" - comprador: {self.comprador}" if self.comprador else ""
        return f"Ingresso {self.id} | {self.status}{comprador_txt}"

In [24]:

class SistemaEventos:
    def __init__(self):
        self.eventos = []
        self.participantes = []
        self.ingressos = []

    # cadastrar retorna o objeto Evento
    def cadastrar_evento(self, nome, data, local, descricao, capacidade):
        evento = Evento(nome, data, local, descricao, capacidade)
        self.eventos.append(evento)
        print(f"Evento cadastrado: {evento.nome} (id: {evento.id})")
        return evento

    # buscar por id
    def buscar_evento(self, id_evento):
        for e in self.eventos:
            if e.id == id_evento:
                return e
        return None


    def consultar_evento(self, chave):
        if isinstance(chave, Evento):
            return chave
        if isinstance(chave, str):
            for e in self.eventos:
                if e.id == chave or e.id.startswith(chave):
                    return e
            for e in self.eventos:
                if e.nome.lower() == chave.lower():
                    return e
        print("Evento não encontrado.")
        return None

    # listar eventos
    def listar_eventos(self):
        if not self.eventos:
            print("Nenhum evento cadastrado.")
            return
        for e in self.eventos:
            print(e)

    # cadastrar participante (associa ao objeto Evento)
    def cadastrar_participante(self, nome, cpf, email, telefone, id_evento):
        evento = self.buscar_evento(id_evento)
        if not evento:
            print("Evento não encontrado. Não foi possível cadastrar participante.")
            return None

        participante = Participante(nome, cpf, email, telefone, evento)
        success = evento.add_participante(participante)
        if not success:
            print("Não foi possível adicionar participante (evento lotado).")
            return None

        self.participantes.append(participante)
        print(f"Participante {participante.nome} cadastrado no evento {evento.nome}.")
        return participante

    # gerar ingresso
    def gerar_ingresso(self, id_evento):
        evento = self.buscar_evento(id_evento)
        if not evento:
            print("Evento não encontrado. Não foi possível gerar ingresso.")
            return None

        ingresso = Ingresso(evento.id)
        evento.gerar_ingresso(ingresso)
        self.ingressos.append(ingresso)
        print(f"Ingresso {ingresso.id} gerado para o evento {evento.nome}.")
        return ingresso

    # vender ingresso
    def vender_ingresso(self, id_ingresso, comprador):
        for i in self.ingressos:
            if i.id == id_ingresso or i.id.startswith(id_ingresso):
                success = i.vender(comprador)
                if success:
                    print(f"Ingresso {i.id} vendido para {comprador}.")
                return i if success else None
        print("Ingresso não encontrado.")
        return None

    # cancelar ingresso
    def cancelar_ingresso(self, id_ingresso):
        for i in self.ingressos:
            if i.id == id_ingresso or i.id.startswith(id_ingresso):
                success = i.cancelar()
                if success:
                    print(f"Ingresso {i.id} cancelado.")
                return i if success else None
        print("Ingresso não encontrado.")
        return None

    # relatório por id_evento
    def relatorio(self, id_evento):
        evento = self.buscar_evento(id_evento)
        if not evento:
            print("Evento não encontrado.")
            return None

        print("\n--- RELATÓRIO DO EVENTO ---")
        print(f"Nome: {evento.nome}")
        print(f"Data: {evento.data}")
        print(f"Local: {evento.local}")
        print(f"Capacidade: {evento.capacidade_maxima}")
        print(f"Ocupação: {evento.ocupacao()}")
        print("\nParticipantes:")
        for p in evento.participantes:
            print(f"- {p.nome} ({p.cpf})")

        print("\nIngressos:")
        for ing in evento.ingressos:
            print(f"- {ing.id} | {ing.status} | comprador: {ing.comprador}\n")
        return {
            "nome": evento.nome,
            "data": evento.data,
            "local": evento.local,
            "capacidade": evento.capacidade_maxima,
            "ocupacao": evento.ocupacao(),
            "participantes": [p.nome for p in evento.participantes],
            "ingressos": [f"{i.id} - {i.status}" for i in evento.ingressos]
        }


In [ ]:
def menu():
    sistema = SistemaEventos()

    while True:
        print("\n========= SISTEMA DE EVENTOS =========")
        print("1 - Cadastrar Evento")
        print("2 - Listar Eventos")
        print("3 - Consultar Evento")
        print("4 - Cadastrar Participante")
        print("5 - Gerar Ingresso")
        print("6 - Vender Ingresso")
        print("7 - Cancelar Ingresso")
        print("8 - Relatório do Evento")
        print("0 - Sair")
        print("======================================")

        opcao = input("Escolha uma opção: ")


        # 1 - CADASTRAR EVENTO
        # ---------------------------- #
        if opcao == "1":
            print("\n=== CADASTRAR EVENTO ===")
            nome = input("Nome do evento: ")
            data = input("Data: ")
            local = input("Local: ")
            descricao = input("Descrição: ")
            capacidade = int(input("Capacidade máxima: "))

            sistema.cadastrar_evento(nome, data, local, descricao, capacidade)


        # 2 - LISTAR EVENTOS
        # ---------------------------- #
        elif opcao == "2":
            print("\n=== LISTA DE EVENTOS ===")
            sistema.listar_eventos()


        # 3 - CONSULTAR EVENTO
        # ---------------------------- #
        elif opcao == "3":
            print("\n=== CONSULTAR EVENTO ===")
            chave = input("Digite ID ou nome do evento: ")
            evento = sistema.consultar_evento(chave)
            if evento:
                print("\nEvento encontrado:")
                print(evento)


        # 4 - CADASTRAR PARTICIPANTE
        # ---------------------------- #
        elif opcao == "4":
            print("\n=== CADASTRAR PARTICIPANTE ===")
            nome = input("Nome: ")
            cpf = input("CPF: ")
            email = input("Email: ")
            telefone = input("Telefone: ")
            id_evento = input("ID do evento: ")

            sistema.cadastrar_participante(nome, cpf, email, telefone, id_evento)


        # 5 - GERAR INGRESSO
        # ---------------------------- #
        elif opcao == "5":
            print("\n=== GERAR INGRESSO ===")
            id_evento = input("ID do evento: ")
            sistema.gerar_ingresso(id_evento)


        # 6 - VENDER INGRESSO
        # ---------------------------- #
        elif opcao == "6":
            print("\n=== VENDER INGRESSO ===")
            id_ingresso = input("ID do ingresso: ")
            comprador = input("Nome do comprador: ")
            sistema.vender_ingresso(id_ingresso, comprador)


        # 7 - CANCELAR INGRESSO
        # ---------------------------- #
        elif opcao == "7":
            print("\n=== CANCELAR INGRESSO ===")
            id_ingresso = input("ID do ingresso: ")
            sistema.cancelar_ingresso(id_ingresso)


        # 8 - RELATÓRIO DO EVENTO
        # ---------------------------- #
        elif opcao == "8":
            print("\n=== RELATÓRIO DO EVENTO ===")
            id_evento = input("ID do evento: ")
            sistema.relatorio(id_evento)


        # 0 - SAIR
        # ---------------------------- #
        elif opcao == "0":
            print("Encerrando o sistema...")

            break

        else:
            print("Opção inválida! Tente novamente.")


# Executar o menu
menu()


========= SISTEMA DE EVENTOS =========
1 - Cadastrar Evento
2 - Listar Eventos
3 - Consultar Evento
4 - Cadastrar Participante
5 - Gerar Ingresso
6 - Vender Ingresso
7 - Cancelar Ingresso
8 - Relatório do Evento
0 - Sair
Escolha uma opção: 1

=== CADASTRAR EVENTO ===
Nome do evento: AEGTAWGHA
Data: 12225
Local: EGASG
Descrição: ASHAFFDH
Capacidade máxima: 10
Evento cadastrado: AEGTAWGHA (id: a140b744-9977-4517-b974-9778c6f94b69)

========= SISTEMA DE EVENTOS =========
1 - Cadastrar Evento
2 - Listar Eventos
3 - Consultar Evento
4 - Cadastrar Participante
5 - Gerar Ingresso
6 - Vender Ingresso
7 - Cancelar Ingresso
8 - Relatório do Evento
0 - Sair
Escolha uma opção: 2

=== LISTA DE EVENTOS ===
Evento: AEGTAWGHA (id: a140b744-9977-4517-b974-9778c6f94b69) - Data: 12225 - Local: EGASG

========= SISTEMA DE EVENTOS =========
1 - Cadastrar Evento
2 - Listar Eventos
3 - Consultar Evento
4 - Cadastrar Participante
5 - Gerar Ingresso
6 - Vender Ingresso
7 - Cancelar Ingresso
8 - Relatório do E